In [17]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [18]:
from glob import glob
from tqdm import tqdm
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from shutil import copy
from sklearn.model_selection import StratifiedKFold
import torch
from tempfile import TemporaryDirectory

from albk.data.utils import idx_to_locate
use_disjoint_files = False


import torch
import torch.nn as nn

from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
from joblib import Parallel, delayed
from itertools import product
import torch
import torch.nn as nn
from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from itertools import product
from torch.utils.data import TensorDataset, DataLoader
from astra.torch.models import EfficientNetClassifier,EfficientNet_B0_Weights
from astra.torch.utils import train_fn

import torchvision.models as models
from astra.torch.metrics import accuracy_score, f1_score, precision_score, recall_score,classification_report

In [19]:
# images_path = expanduser("/home/rishabh.mondal/bkdb/statewise/West_Bengal/")
images_path = expanduser('/home/rishabh.mondal/bkdb/districtwise/birbhum/')
zarr_files = [f for f in os.listdir(images_path) if f.endswith(".zarr")]


In [20]:
files = []
for zarr_file in zarr_files:
    image_path = join(images_path, zarr_file)
    files.append(image_path)

print(len(files))
print(files[:5])

4031
['/home/rishabh.mondal/bkdb/districtwise/birbhum/24.23,87.97.zarr', '/home/rishabh.mondal/bkdb/districtwise/birbhum/23.84,87.83.zarr', '/home/rishabh.mondal/bkdb/districtwise/birbhum/23.94,87.50.zarr', '/home/rishabh.mondal/bkdb/districtwise/birbhum/23.81,87.44.zarr', '/home/rishabh.mondal/bkdb/districtwise/birbhum/23.68,87.57.zarr']


In [21]:
files = []

def get_index_and_image(images_path):
    images = []
    index = []

    # for zarr_file in zarr_files:
    zarr_files = [f for f in os.listdir(images_path) if f.endswith(".zarr")]
    for zarr_file in zarr_files:
        image_path = join(images_path, zarr_file)
        # print(image_path)
        files.append(image_path)
        image_ds = xr.open_zarr(image_path, consolidated=False)
        for lat_lag, lon_lag in product(range(-2, 3), repeat=2):
            images.append(torch.tensor(image_ds.sel(lat_lag=lat_lag, lon_lag=lon_lag)['data'].values)[np.newaxis, ...])
            index.append((zarr_file, lat_lag, lon_lag))
    return images


In [22]:
images = get_index_and_image(images_path)


In [ ]:
print(images[0].shape)

torch.Size([1, 224, 224, 3])


In [ ]:
concatenated_images = torch.cat(images)
concatenated_images= concatenated_images.permute(0,3,2,1)
print(concatenated_images.shape)


torch.Size([100775, 3, 224, 224])


In [ ]:
concatenated_images.dtype

torch.uint8

In [ ]:

# save_path="/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/west_bengal.pt"
# torch.save({
#     'images': concatenated_images
# }, save_path)

In [27]:
images1=concatenated_images
images1 = images1 / 255
    # mean normalize
images1 = (images1 - images1.mean(dim=(0, 2, 3), keepdim=True)) / images1.std(dim=(0, 2, 3), keepdim=True)
images1.shape


torch.Size([100775, 3, 224, 224])

In [28]:
labels = torch.zeros(len(images1))

testset=TensorDataset(images1,labels)
testloader=DataLoader(testset,batch_size=512,shuffle=False)


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
import torch
import torchvision

In [35]:
path='/home/vannsh.jani/brick_kilns/githubrepo/ML/model_50_no_ssl_features_imagenet.pth'

path2='/home/vannsh.jani/brick_kilns/githubrepo/ML/model_50_no_ssl_classifier_imagenet.pth'
model = torchvision.models.efficientnet_b0(pretrained=False)
model.classifier = nn.Linear(1280,2)
model.features.load_state_dict(torch.load(path))
model.classifier.load_state_dict(torch.load(path2))

<All keys matched successfully>

In [ ]:
# train_model = EfficientNetClassifier(
#     models.efficientnet_b0,None, n_classes=2, activation=nn.ReLU(), dropout=0.1
# ).to(device)

In [ ]:
# train_model.load_state_dict(torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/delhi_whole_data_pretrain_aug.pt"))

<All keys matched successfully>

In [ ]:
from torchvision import transforms
model = model.to(device)

In [36]:
# train_model='/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/delhi_whole_data_train_model.pt'
positive_image_indices = []
save_dir = '/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/predicted_positive/jhajhar/'
os.makedirs(save_dir, exist_ok=True)

# # # with torch.no_grad():
# # #     pred_classes =model.predict_class(
# # #         dataloader=testloader, batch_size=512, verbose=True
# # #     ).to(device)

# # pred_classes = torch.zeros(len(images1))
# # for i, (inputs, labels) in enumerate(testloader):
# #     inputs, labels = inputs.to(device), labels.to(device)
# #     outputs = model(inputs)
# #     _, predicted = torch.max(outputs, 1)
# #     pred_classes[i*512:(i+1)*512] = predicted
# # zero_predictions = torch.sum(pred_classes == 0)
# one_predictions = torch.sum(pred_classes == 1)
with torch.no_grad():
    model.eval()
    pred_classes = torch.zeros(len(images1))
    for i

print("Number of zero predictions:", zero_predictions.item())
print("Number of one predictions:", one_predictions.item())
images=images1
for i in range(len(images)):
    # Check if the prediction is positive (class 1)
    if pred_classes[i] == 1:
        normalized_image = (images[i] - images[i].min()) / (images[i].max() - images[i].min())
        pil_image = transforms.ToPILImage()(normalized_image.cpu())

        # append the index of the positive image
        positive_image_indices.append(i)
        
        image_name = f"positive_image_{i}.jpg"
        image_path = os.path.join(save_dir, image_name)
        pil_image.save(image_path)            

# Print the list of positive image indices
print(positive_image_indices)
print(len(positive_image_indices))

AttributeError: 'EfficientNet' object has no attribute 'predict_class'

: 